In [10]:
import requests, bs4
import mysql.connector
from lxml import html
from urllib.request import Request,urlopen
from urllib.parse import urlencode,quote_plus
from bs4 import BeautifulSoup
from xml.etree.ElementTree import parse
from datetime import datetime
from ServiceKey import public_serviceKey, mySQL_Connect

mydb = mySQL_Connect()
# 2021.09.19 insert 에러
cursor = mydb.cursor(prepared=True)
citycode = '31040'

query1 = ("select citycode from public_busctycodelist where citycode >= {}".format(citycode))

cursor.execute(query1)

location_list1 = []

for a in cursor:
    location_list1.append(a)
    
b_cnt = 0
for z in location_list1:
    if b_cnt == 1:
        break
        
    location = z
    query2 = ("select distinct routeid from public_busroute_info where citycode = '{}'".format(location[0]))
    cursor.execute(query2)

    location_list2 = []

    for b in cursor:
        location_list2.append(b)

    now = datetime.now()
    nowDate = now.strftime('%Y%m%d')

    cnt = 1
    for i in location_list2:
        routeId = i

        characters = "(',')"

        join_data = ''.join( x for x in routeId if x not in characters)
        print("조회날짜 : {} | location : {} | cnt : {} | start {} public_busroute_info".format(nowDate,location[0],cnt,join_data))

        url = 'http://openapi.tago.go.kr/openapi/service/BusRouteInfoInqireService/getRouteAcctoThrghSttnList'
        queryParams = '?' + urlencode({
            quote_plus('serviceKey') : public_serviceKey(),
            quote_plus('cityCode') : citycode,
            quote_plus('pageNo') : '1',
            quote_plus('numOfRows') : '10000',
            quote_plus('routeId') : join_data
        })

        response = requests.get(url + queryParams).text.encode('utf-8')
        xmlobj = bs4.BeautifulSoup(response, 'lxml-xml')

        rows = xmlobj.findAll('item')
        resultCode = xmlobj.findAll('resultCode')
        if resultCode[0].text == '99':
            print("location {} | {} 번째부터 조회수 제한이 걸렸습니다.".format(location[0],join_data))
            mydb.close()
            b_cnt += 1
            break
        cnt += 1
        for j in rows:
            item = j
            lat = item.findAll('gpslati')
            lng = item.findAll('gpslong')
            nodeid = item.findAll('nodeid')
            nodenm = item.findAll('nodenm')
            nodeno = item.findAll('nodeno')
            nodeord = item.findAll('nodeord')
            routeid = item.findAll('routeid')
            updowncd = item.findAll('updowncd')
            try:
                if updowncd[0].text == '0':
                    updowncd = '상행'
                elif updowncd[0].text == '1':
                    updowncd = '하행'
            except IndexError:
                updowncd = None

            try:
                sql = "INSERT INTO public_busroute_detail (lat, lng, nodeid, nodenm, nodeno, nodeord, routeid, updowncd,citycode) VALUES (?,?,?,?,?,?,?,?,?)"
                val = (lat[0].text, lng[0].text, nodeid[0].text, nodenm[0].text, nodeno[0].text, nodeord[0].text, routeid[0].text, updowncd,citycode)
            except IndexError:
                sql = "INSERT INTO public_busroute_detail (lat, lng, nodeid, nodenm, nodeno, nodeord, routeid, updowncd,citycode) VALUES (?,?,?,?,?,?,?,?,?)"
                val = (lat[0].text, lng[0].text, nodeid[0].text, nodenm[0].text, None, nodeord[0].text, routeid[0].text, updowncd,citycode)

            cursor.execute(sql, val)
            mydb.commit()

조회날짜 : 20210919 | location : 31030 | cnt : 1 | start GGB207000004 public_busroute_info
location 31030 | GGB207000004 번째부터 조회수 제한이 걸렸습니다.
